In [46]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import KFold, StratifiedKFold
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier

from itertools import combinations
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier


In [52]:
def process_data(data):
    '''
    remove redundant columns
    '''
    #rems = ['Id', 'Soil_Type7', 'Soil_Type8', 'Soil_Type15', 'Soil_Type25']
    rems = ['Id', 'Soil_Type7', 'Soil_Type15']
    #rems = ['Id']
#     #Add constant columns as they don't help in prediction process
#     for c in data.columns:
#         if data[c].std() == 0: #standard deviation is zero
#             rem.append(c)

    #drop the columns
    for rem in rems:
        data.drop(rem,axis=1,inplace=True)
    

    return data

In [3]:
def score(y, y_pred):

    y_true = np.array(y, dtype=int)
    y_predict = np.array(y_pred, dtype=int)
    
    from sklearn.metrics import f1_score

    return f1_score(y_true, y_predict, average='micro')

In [59]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer

def normalize_train_data(dataset):
    r, c = dataset.shape
    array = dataset.values
    X_all = array[:,0:(c-1)]
    y_all = array[:,(c-1)]
    size = 10
    X_num = X_all[:,0:size]
    X_cat = X_all[:,size:]

    X_num = StandardScaler().fit_transform(X_num)
    #X_num = MinMaxScaler().fit_transform(X_num)
    X_num = Normalizer().fit_transform(X_num)

    X_all_scaled = np.concatenate((X_num, X_cat), axis=1)
    
    return X_all_scaled, y_all

def normalize_test_data(dataset):
    r, c = dataset.shape
    X_all = dataset.values
    y_all = []
    size = 10
    X_num = X_all[:,0:size]
    X_cat = X_all[:,size:]

    X_num = StandardScaler().fit_transform(X_num)
    #X_num = MinMaxScaler().fit_transform(X_num)
    X_num = Normalizer().fit_transform(X_num)

    X_all_scaled = np.concatenate((X_num, X_cat), axis=1)
    
    return X_all_scaled, y_all

def train_extract(train, test):
    X_train, y_train = normalize_train_data(train)
    X_test, y_test = normalize_train_data(test)
    
    return X_train, y_train, X_test, y_test



In [60]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV

def tuner(model, param_grid, dataset):
    X_train, y_train, _, _ = train_extract(dataset, dataset)
    tuning_scorer = make_scorer(score, greater_is_better = True)
    
    tuner_model = GridSearchCV(estimator=model, 
                                param_grid=param_grid, 
                                scoring=tuning_scorer,
                                verbose=10, 
                                n_jobs=-1, 
                                iid=True, 
                                refit=True,
                                cv=5)

    tuner_model.fit(X_train, y_train)
    print("Best score: %0.3f" % tuner_model.best_score_)
    print("Best parameters set:")
    best_parameters = tuner_model.best_estimator_.get_params()
    for param_name in sorted(param_grid.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))


In [61]:
train_raw = pd.read_csv('data/train.csv')
msk = np.random.rand(len(train_raw)) < 0.9
train = train_raw[msk]
validation = train_raw[~msk]

train_clean = process_data(train)
validation_clean = process_data(validation)

X_train, y_train, X_test, y_test = train_extract(train_clean, validation_clean)


/Users/brandon/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/brandon/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


In [8]:
model = KNeighborsClassifier(n_jobs=-1)
param_grid = {
        #'n_estimators': np.arange(89, 100, 2),
        'algorithm':["auto", "ball_tree", "kd_tree", "brute"],
        'n_neighbors':[1,2,3]
    }

tuner(model, param_grid, train_clean)



Fitting 5 folds for each of 12 candidates, totalling 60 fits


/Users/brandon/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


[CV] algorithm=auto, n_neighbors=1 ...................................
[CV] algorithm=auto, n_neighbors=1 ...................................
[CV] algorithm=auto, n_neighbors=1 ...................................
[CV] algorithm=auto, n_neighbors=1 ...................................
[CV] .......... algorithm=auto, n_neighbors=1, score=0.691517 -   1.2s
[CV] algorithm=auto, n_neighbors=1 ...................................
[CV] .......... algorithm=auto, n_neighbors=1, score=0.698238 -   1.1s
[CV] algorithm=auto, n_neighbors=2 ...................................
[CV] .......... algorithm=auto, n_neighbors=1, score=0.715963 -   1.1s
[CV] algorithm=auto, n_neighbors=2 ...................................
[CV] .......... algorithm=auto, n_neighbors=1, score=0.733284 -   1.2s
[CV] algorithm=auto, n_neighbors=2 ...................................
[CV] .......... algorithm=auto, n_neighbors=1, score=0.798088 -   0.7s
[CV] algorithm=auto, n_neighbors=2 ...................................


[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   10.6s


[CV] .......... algorithm=auto, n_neighbors=2, score=0.654185 -   1.1s
[CV] algorithm=auto, n_neighbors=2 ...................................
[CV] .......... algorithm=auto, n_neighbors=2, score=0.663239 -   1.3s
[CV] algorithm=auto, n_neighbors=3 ...................................
[CV] .......... algorithm=auto, n_neighbors=2, score=0.672294 -   0.9s
[CV] algorithm=auto, n_neighbors=3 ...................................
[CV] .......... algorithm=auto, n_neighbors=2, score=0.692873 -   1.1s
[CV] algorithm=auto, n_neighbors=3 ...................................
[CV] .......... algorithm=auto, n_neighbors=2, score=0.774549 -   0.9s
[CV] algorithm=auto, n_neighbors=3 ...................................


...

[CV] ......... algorithm=brute, n_neighbors=3, score=0.669358 - 1.3min
[CV] ......... algorithm=brute, n_neighbors=3, score=0.662996 - 1.2min
[CV] algorithm=brute, n_neighbors=3 ..................................


/Users/brandon/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


[CV] ......... algorithm=brute, n_neighbors=3, score=0.657363 -   9.8s
[CV] ......... algorithm=brute, n_neighbors=3, score=0.695077 -   9.1s
[CV] ......... algorithm=brute, n_neighbors=3, score=0.776388 -   9.7s


[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 25.4min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 25.4min finished


Best score: 0.727
Best parameters set:
	algorithm: 'auto'
	n_neighbors: 1


In [62]:
model = KNeighborsClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
result = score(y_test, y_pred)
print(result)

0.783518639634


In [63]:
model = KNeighborsClassifier(n_jobs=-1, algorithm='auto', n_neighbors=1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
result = score(y_test, y_pred)
print(result)

0.817527795945


In [29]:
seed = 19
model = DecisionTreeClassifier(random_state=seed)
param_grid = {
    'max_depth': np.arange(10, 30, 2),
    }
tuner(model, param_grid, train_clean)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/Users/brandon/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


[CV] max_depth=10 ....................................................
[CV] max_depth=10 ....................................................
[CV] max_depth=10 ....................................................
[CV] max_depth=10 ....................................................
[CV] ........................... max_depth=10, score=0.642489 -   0.0s
[CV] ........................... max_depth=10, score=0.698489 -   0.0s
[CV] ........................... max_depth=10, score=0.622010 -   0.0s
[CV] max_depth=10 ....................................................
[CV] ........................... max_depth=10, score=0.658195 -   0.0s
[CV] max_depth=12 ....................................................
[CV] max_depth=12 ....................................................
[CV] max_depth=12 ....................................................
[CV] ........................... max_depth=10, score=0.713021 -   0.0s
[CV] ........................... max_depth=12, score=0.660287 -   0.0s
[CV] .

[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.9s


[CV] ........................... max_depth=12, score=0.703649 -   0.0s
[CV] ........................... max_depth=12, score=0.749170 -   0.0s
[CV] max_depth=14 ....................................................
[CV] ........................... max_depth=14, score=0.674273 -   0.0s
[CV] max_depth=14 ....................................................
[CV] max_depth=14 ....................................................
[CV] ........................... max_depth=14, score=0.671208 -   0.0s
[CV] max_depth=16 ....................................................


...

[CV] ........................... max_depth=24, score=0.690835 -   0.0s
[CV] max_depth=24 ....................................................
[CV] ........................... max_depth=24, score=0.663844 -   0.0s
[CV] max_depth=26 ....................................................
[CV] ........................... max_depth=24, score=0.698489 -   0.0s
[CV] ........................... max_depth=24, score=0.654512 -   0.0s
[CV] max_depth=26 ....................................................
[CV] max_depth=26 ....................................................
[CV] ........................... max_depth=24, score=0.763925 -   0.0s
[CV] max_depth=26 ....................................................
[CV] ........................... max_depth=26, score=0.691572 -   0.0s
[CV] max_depth=26 ....................................................
[CV] ........................... max_depth=26, score=0.657459 -   0.0s
[CV] max_depth=28 ....................................................
[CV] .

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    4.7s


[CV] ........................... max_depth=26, score=0.696277 -   0.0s
[CV] max_depth=28 ....................................................
[CV] ........................... max_depth=26, score=0.769089 -   0.0s
[CV] max_depth=28 ....................................................
[CV] ........................... max_depth=28, score=0.686787 -   0.0s
[CV] max_depth=28 ....................................................
[CV] ........................... max_depth=28, score=0.668630 -   0.0s
[CV] ........................... max_depth=28, score=0.657459 -   0.0s
[CV] ........................... max_depth=28, score=0.694434 -   0.0s
[CV] ........................... max_depth=28, score=0.769458 -   0.0s


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    5.4s finished


Best score: 0.697
Best parameters set:
	max_depth: 26


In [64]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
result = score(y_test, y_pred)
print(result)

0.752125572269


In [66]:
seed = 19
model = DecisionTreeClassifier(random_state=seed, max_depth=14)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
result = score(y_test, y_pred)
print(result)

0.75408763898


In [36]:
from sklearn.ensemble import BaggingClassifier

base_estimator = DecisionTreeClassifier(random_state=seed,max_depth=14)
model = BaggingClassifier(n_jobs=-1,base_estimator=base_estimator, random_state=seed)
param_grid = {
    'n_estimators': np.arange(80, 100, 2),
    }
tuner(model, param_grid, train_clean)


Fitting 5 folds for each of 10 candidates, totalling 50 fits


/Users/brandon/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


[CV] n_estimators=80 .................................................
[CV] n_estimators=80 .................................................
[CV] n_estimators=80 .................................................


/Users/brandon/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


[CV] n_estimators=80 .................................................


...

[CV] ........................ n_estimators=98, score=0.710352 -   0.2s
[CV] ........................ n_estimators=98, score=0.699596 -   0.3s


/Users/brandon/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/Users/brandon/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


[CV] ........................ n_estimators=98, score=0.751286 -   0.2s
[CV] ........................ n_estimators=98, score=0.798823 -   0.2s


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  3.3min finished


Best score: 0.737
Best parameters set:
	n_estimators: 98


In [39]:
base_estimator = DecisionTreeClassifier(random_state=seed,max_depth=11)
model = BaggingClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
result = score(y_test, y_pred)
print(result)

0.786175710594


In [41]:
base_estimator = DecisionTreeClassifier(random_state=seed,max_depth=11)
model = BaggingClassifier(n_jobs=-1,base_estimator=base_estimator, n_estimators=300, random_state=seed)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
result = score(y_test, y_pred)
print(result)

0.821499668215


In [16]:
from sklearn.svm import SVC
model = SVC(random_state=seed)
param_grid = {
    #'kernel': [‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’ ],
    'C': np.arange(1, 11, 1),
    }
tuner(model, param_grid, train_clean)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/Users/brandon/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


[CV] C=1 .............................................................
[CV] C=1 .............................................................
[CV] C=1 .............................................................
[CV] C=1 .............................................................
[CV] .................................... C=1, score=0.492477 -   3.1s
[CV] C=1 .............................................................
[CV] .................................... C=1, score=0.598972 -   3.2s
[CV] C=2 .............................................................
[CV] .................................... C=1, score=0.548458 -   3.2s
[CV] C=2 .............................................................
[CV] .................................... C=1, score=0.603600 -   3.2s
[CV] C=2 .............................................................
[CV] .................................... C=2, score=0.497615 -   2.8s
[CV] C=2 .............................................................


[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   57.5s


[CV] .................................... C=2, score=0.549559 -   2.8s
[CV] .................................... C=2, score=0.597136 -   2.9s
[CV] C=2 .............................................................
[CV] C=3 .............................................................
[CV] .................................... C=1, score=0.621184 -   3.0s
[CV] C=3 .............................................................
[CV] .................................... C=3, score=0.502752 -   2.7s
[CV] C=3 .............................................................
[CV] .................................... C=2, score=0.602131 -   2.9s
[CV] C=3 .............................................................


...

[CV] .................................... C=7, score=0.624173 -   3.0s
[CV] C=8 .............................................................
[CV] .................................... C=7, score=0.657227 -   2.9s
[CV] C=8 .............................................................
[CV] .................................... C=8, score=0.533945 -   2.7s
[CV] C=8 .............................................................
[CV] .................................... C=8, score=0.548825 -   3.0s
[CV] C=9 .............................................................
[CV] .................................... C=8, score=0.626148 -   3.5s
[CV] C=9 .............................................................
[CV] .................................... C=8, score=0.627112 -   3.4s
[CV] C=9 .............................................................
[CV] .................................... C=8, score=0.661273 -   3.4s
[CV] C=9 .............................................................
[CV] .

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  5.4min


[CV] .................................... C=9, score=0.630555 -   2.9s
[CV] C=10 ............................................................
[CV] .................................... C=9, score=0.628215 -   2.9s
[CV] C=10 ............................................................
[CV] .................................... C=9, score=0.664215 -   2.7s
[CV] C=10 ............................................................
[CV] ................................... C=10, score=0.540917 -   2.6s
[CV] C=10 ............................................................
[CV] ................................... C=10, score=0.552496 -   2.6s
[CV] ................................... C=10, score=0.633492 -   2.7s
[CV] ................................... C=10, score=0.626745 -   2.0s
[CV] ................................... C=10, score=0.669732 -   1.9s


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  6.0min finished


Best score: 0.605
Best parameters set:
	C: 10


In [41]:
model = SVC()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
result = score(y_test, y_pred)
print(result)


0.614341085271


In [17]:
model = SVC(random_state=seed, C=10)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
result = score(y_test, y_pred)
print(result)


0.646317186463


In [19]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_jobs=-1, random_state=seed)
param_grid = {
    'n_estimators': np.arange(10, 20, 2),
    }
tuner(model, param_grid, train_clean)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


/Users/brandon/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


[CV] n_estimators=10 .................................................
[CV] n_estimators=10 .................................................
[CV] n_estimators=10 .................................................
[CV] n_estimators=10 .................................................
[CV] ........................ n_estimators=10, score=0.698349 -   0.1s
[CV] ........................ n_estimators=10, score=0.696035 -   0.1s
[CV] n_estimators=10 .................................................
[CV] ........................ n_estimators=10, score=0.732550 -   0.1s
[CV] ........................ n_estimators=10, score=0.708777 -   0.1s
[CV] n_estimators=12 .................................................
[CV] n_estimators=12 .................................................
[CV] n_estimators=12 .................................................
[CV] ........................ n_estimators=10, score=0.806547 -   0.2s
[CV] n_estimators=12 .................................................
[CV] .

[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    1.7s


[CV] ........................ n_estimators=12, score=0.736591 -   0.2s
[CV] n_estimators=14 .................................................
[CV] ........................ n_estimators=14, score=0.705321 -   0.1s
[CV] ........................ n_estimators=12, score=0.808753 -   0.1s
[CV] ........................ n_estimators=14, score=0.699339 -   0.1s
[CV] n_estimators=14 .................................................
[CV] n_estimators=14 .................................................
[CV] n_estimators=16 .................................................


...

[CV] n_estimators=18 .................................................
[CV] n_estimators=18 .................................................
[CV] ........................ n_estimators=18, score=0.708257 -   0.2s
[CV] n_estimators=18 .................................................
[CV] ........................ n_estimators=18, score=0.708884 -   0.1s
[CV] ........................ n_estimators=18, score=0.730444 -   0.1s
[CV] ........................ n_estimators=18, score=0.754592 -   0.1s


[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed:    5.5s remaining:    1.1s


[CV] ........................ n_estimators=18, score=0.816109 -   0.1s


[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    6.2s finished


Best score: 0.744
Best parameters set:
	n_estimators: 18


In [42]:
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
result = score(y_test, y_pred)
print(result)

0.782945736434


In [20]:
model = RandomForestClassifier(n_jobs=-1, n_estimators=18, random_state=seed)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
result = score(y_test, y_pred)
print(result)

0.830789648308


In [23]:
from sklearn.ensemble import ExtraTreesClassifier

model = ExtraTreesClassifier(n_jobs=-1, random_state=seed)
param_grid = {
    'n_estimators': np.arange(10, 24, 2),
    }
tuner(model, param_grid, train_clean)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


/Users/brandon/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


[CV] n_estimators=10 .................................................
[CV] n_estimators=10 .................................................
[CV] n_estimators=10 .................................................
[CV] n_estimators=10 .................................................
[CV] ........................ n_estimators=10, score=0.720367 -   0.2s
[CV] n_estimators=10 .................................................
[CV] ........................ n_estimators=10, score=0.704479 -   0.2s
[CV] n_estimators=12 .................................................
[CV] ........................ n_estimators=10, score=0.722732 -   0.1s
[CV] ........................ n_estimators=10, score=0.748714 -   0.1s
[CV] n_estimators=12 .................................................
[CV] n_estimators=12 .................................................
[CV] ........................ n_estimators=10, score=0.820890 -   0.2s
[CV] n_estimators=12 .................................................
[CV] .

[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    1.3s


[CV] ........................ n_estimators=12, score=0.721835 -   0.1s
[CV] ........................ n_estimators=12, score=0.705580 -   0.1s
[CV] n_estimators=14 .................................................
[CV] n_estimators=14 .................................................
[CV] ........................ n_estimators=12, score=0.754225 -   0.1s
[CV] n_estimators=14 .................................................
[CV] ........................ n_estimators=12, score=0.822361 -   0.1s
[CV] n_estimators=14 .................................................
[CV] ........................ n_estimators=14, score=0.711454 -   0.1s


...

[CV] n_estimators=20 .................................................
[CV] ........................ n_estimators=20, score=0.721366 -   0.1s
[CV] n_estimators=22 .................................................
[CV] ........................ n_estimators=20, score=0.766716 -   0.2s
[CV] n_estimators=22 .................................................
[CV] ........................ n_estimators=20, score=0.734484 -   0.1s
[CV] ........................ n_estimators=20, score=0.828981 -   0.1s
[CV] n_estimators=22 .................................................
[CV] n_estimators=22 .................................................
[CV] ........................ n_estimators=22, score=0.733945 -   0.1s
[CV] n_estimators=22 .................................................
[CV] ........................ n_estimators=22, score=0.716960 -   0.2s


[Parallel(n_jobs=-1)]: Done  32 out of  35 | elapsed:    6.6s remaining:    0.6s


[CV] ........................ n_estimators=22, score=0.767083 -   0.1s
[CV] ........................ n_estimators=22, score=0.739625 -   0.1s
[CV] ........................ n_estimators=22, score=0.830085 -   0.1s


[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed:    7.0s finished


Best score: 0.758
Best parameters set:
	n_estimators: 22


In [43]:
model = ExtraTreesClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
result = score(y_test, y_pred)
print(result)

0.826873385013


In [24]:
model = ExtraTreesClassifier(n_jobs=-1, n_estimators=22, random_state=seed)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
result = score(y_test, y_pred)
print(result)

0.829462508295


In [25]:
model = SGDClassifier(loss='log',penalty='elasticnet', random_state=seed, shuffle=True)
     
param_grid = {
    #'loss':[ "hinge", "log", "modified_huber", "squared_hinge", "perceptron"],
    #'penalty': ["l2", "l1", "elasticnet"],
    'n_iter': [1,2,3,4,5,6,7,8,9,20]
    }
    
tuner(model, param_grid, train_clean)

/Users/brandon/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] n_iter=1 ........................................................
[CV] n_iter=1 ........................................................
[CV] n_iter=1 ........................................................
[CV] n_iter=1 ........................................................
[CV] ............................... n_iter=1, score=0.429883 -   0.0s
[CV] ............................... n_iter=1, score=0.524771 -   0.0s
[CV] n_iter=1 ........................................................
[CV] ............................... n_iter=1, score=0.513960 -   0.0s
[CV] ............................... n_iter=1, score=0.560044 -   0.0s
[CV] n_iter=2 ........................................................
[CV] n_iter=2 ........................................................
[CV] n_iter=2 ........................................................
[CV] ............................... n_iter=1, score=0.551673 -   0.0s
[CV] n_iter=2 ..

[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.6s


[CV] n_iter=3 ........................................................
[CV] ............................... n_iter=2, score=0.565760 -   0.0s
[CV] ............................... n_iter=2, score=0.575579 -   0.0s
[CV] n_iter=3 ........................................................
[CV] n_iter=3 ........................................................
[CV] ............................... n_iter=3, score=0.568440 -   0.0s
[CV] ............................... n_iter=3, score=0.546256 -   0.0s
[CV] n_iter=4 ........................................................
[CV] n_iter=3 ........................................................


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    1.0s


...

[CV] n_iter=8 ........................................................
[CV] ............................... n_iter=7, score=0.648032 -   0.0s
[CV] n_iter=8 ........................................................
[CV] ............................... n_iter=8, score=0.568073 -   0.0s
[CV] n_iter=8 ........................................................
[CV] ............................... n_iter=8, score=0.560940 -   0.0s
[CV] n_iter=9 ........................................................
[CV] ............................... n_iter=8, score=0.595299 -   0.0s
[CV] n_iter=9 ........................................................
[CV] ............................... n_iter=8, score=0.602866 -   0.0s
[CV] n_iter=9 ........................................................
[CV] ............................... n_iter=8, score=0.672674 -   0.0s
[CV] n_iter=9 ........................................................
[CV] ............................... n_iter=9, score=0.584220 -   0.0s
[CV] n

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    6.3s


[CV] ............................... n_iter=9, score=0.632256 -   0.0s
[CV] n_iter=20 .......................................................
[CV] ............................... n_iter=9, score=0.627437 -   0.0s
[CV] n_iter=20 .......................................................
[CV] .............................. n_iter=20, score=0.566239 -   0.0s
[CV] n_iter=20 .......................................................
[CV] .............................. n_iter=20, score=0.524963 -   0.0s
[CV] .............................. n_iter=20, score=0.599339 -   0.0s
[CV] .............................. n_iter=20, score=0.627112 -   0.0s
[CV] .............................. n_iter=20, score=0.659066 -   0.0s


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    9.1s finished


Best score: 0.600
Best parameters set:
	n_iter: 8


In [44]:
model = SGDClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
result = score(y_test, y_pred)
print(result)

0.633720930233


In [26]:
model = SGDClassifier(loss='log',penalty='elasticnet', n_iter=8, random_state=seed, shuffle=True)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
result = score(y_test, y_pred)
print(result)

0.654943596549


In [42]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier(random_state=seed)
param_grid = {
    'max_depth': np.arange(1, 10, 1),
    }
tuner(model, param_grid, train_clean)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


/Users/brandon/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


[CV] max_depth=1 .....................................................
[CV] max_depth=1 .....................................................
[CV] max_depth=1 .....................................................
[CV] max_depth=1 .....................................................
[CV] ............................ max_depth=1, score=0.553028 -   0.1s
[CV] max_depth=1 .....................................................
[CV] ............................ max_depth=1, score=0.622842 -   0.0s
[CV] max_depth=2 .....................................................
[CV] ............................ max_depth=1, score=0.615639 -   0.0s
[CV] max_depth=2 .....................................................
[CV] ............................ max_depth=1, score=0.651727 -   0.0s
[CV] max_depth=2 .....................................................
[CV] ............................ max_depth=1, score=0.696947 -   0.0s
[CV] max_depth=2 .....................................................


[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   43.4s


[CV] ............................ max_depth=2, score=0.648807 -   0.0s
[CV] max_depth=2 .....................................................
[CV] ............................ max_depth=2, score=0.656388 -   0.0s
[CV] ............................ max_depth=2, score=0.674624 -   0.0s
[CV] max_depth=3 .....................................................
[CV] max_depth=3 .....................................................
[CV] ............................ max_depth=2, score=0.677076 -   0.0s
[CV] max_depth=3 .....................................................
[CV] ............................ max_depth=2, score=0.749540 -   0.0s
[CV] max_depth=3 .....................................................


...

[CV] ............................ max_depth=7, score=0.754225 -   0.1s
[CV] max_depth=8 .....................................................
[CV] ............................ max_depth=7, score=0.830085 -   0.1s
[CV] max_depth=8 .....................................................
[CV] ............................ max_depth=8, score=0.743853 -   0.2s
[CV] max_depth=8 .....................................................
[CV] ............................ max_depth=8, score=0.719163 -   0.2s
[CV] max_depth=9 .....................................................
[CV] ............................ max_depth=8, score=0.743665 -   0.1s
[CV] max_depth=9 .....................................................
[CV] ............................ max_depth=8, score=0.761205 -   0.2s
[CV] max_depth=9 .....................................................
[CV] ............................ max_depth=8, score=0.840750 -   0.2s
[CV] max_depth=9 .....................................................
[CV] .

[Parallel(n_jobs=-1)]: Done  43 out of  45 | elapsed: 17.3min remaining:   48.4s


[CV] ............................ max_depth=9, score=0.760103 -   0.1s
[CV] ............................ max_depth=9, score=0.835234 -   0.1s


[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed: 18.3min finished


Best score: 0.762
Best parameters set:
	max_depth: 8


In [47]:
model = GradientBoostingClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
result = score(y_test, y_pred)
print(result)

0.766795865633


In [45]:
model = GradientBoostingClassifier(max_depth=8, random_state=seed)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
result = score(y_test, y_pred)
print(result)

0.842733908427
